# Cindy Ho 127008544 User Feedback Prediction Algorithm

In [1]:
import csv #library for importing csv files
import firebase_admin #needed for connecting to database
from firebase_admin import credentials #needed for certifying password to database
from firebase_admin import firestore #needed for admin access to database
import pandas as pd #for pandas dataframe - structure of organizing data
from sklearn.model_selection import train_test_split #used to split dataset into training and testing dataset
# import numpy as np 
from sklearn import neighbors #import library for KNN
from sklearn.ensemble import RandomForestClassifier #import RFClassifier model
from sklearn.naive_bayes import GaussianNB #import GaussianNB model
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score
#imports tools needed to verify results of models

# Connection to Firebase Database

In [2]:
#the purpose of this is to gain access to the database
if not firebase_admin._apps: #using a if not statement to prevent overwriting over same password key
    #saves to cred variable of admin password key
    cred = credentials.Certificate(r"C:\Users\cindy\OneDrive\Documents\403\perfectpourovercoffee-3c0ce-firebase-adminsdk-h2jgk-1590472422.json")
    
    #calls upon firebase admin library and inserts password key to gain access to private database
    default_app = firebase_admin.initialize_app(cred)
    

db = firestore.client() #saves db as the database


# Collects user dataset from CSV file and writes to database

In [3]:
#read from my csv user feedback file and then write it to the database
rows = [] #created a list
with open(r'C:\Users\cindy\OneDrive\Documents\403\UserFeedback.csv','r') as file: #opens csv input 
    csv_reader = csv.reader(file, delimiter=',') #reads from csv file
    for row in csv_reader: #iterates through csv
        rows.append(row) #appends to list while it iterates through csv
        doc_ref = { #generates brew data
            'user_id': row[0], #takes first value and sets as user id value
            'cup_size': row[1], #takes second value and sets as cup size value
            'roast_type': row[2], #takes third value and sets as roast type value
            'coffee_bean_type': row[3], #takes fourth value and sets as coffee bean type value
            'strength': row[4], #takes fifth value and sets as user preferred strength value
            'rating': row[5], #takes sixth value and sets as user rating value
            'temperature': row[6], #takes seventh value and sets as target temperature value
            'grind_size': row[7], #takes eighth value and sets as recommended grind size value
            'water_volume': row[8] #takes ninth value and sets as target water volume value
        }
        db.collection('demobrew').add(doc_ref) #all these datapoints are saved as one collection / folder 
        #i used cindy brews to reference

# Collects user dataset from database & puts into a pandas dataframe for ML

In [4]:
#now reading from database to implement machine learning
docs = db.collection('demobrew').stream() #reads from all documents in a collection of the database
data = pd.DataFrame() #this creates the pandas dataframe containing all of the user feedback from the database
for doc in docs: #iterates through the document stream 
    print(f'{doc.id} => {doc.to_dict()}') #shows programmer what is being saved
    data = data.append(doc.to_dict(),ignore_index=True) # appends it to my pandas dataframe

027lCwTAXBGcT5NkFZD6 => {'rating': '4', 'strength': '3', 'grind_size': 'larger', 'cup_size': '8', 'water_volume': '14', 'user_id': '6', 'roast_type': 'medium', 'temperature': '186', 'coffee_bean_type': 'excelsa'}
1QApglKioyhJzmst5RAm => {'grind_size': 'smaller', 'user_id': '5', 'coffee_bean_type': 'robusta', 'cup_size': '16', 'water_volume': '18', 'rating': '10', 'strength': '2', 'roast_type': 'medium', 'temperature': '190'}
1muQGt5diaYO9YEkvkLy => {'strength': '3', 'coffee_bean_type': 'robusta', 'rating': '9', 'cup_size': '8', 'water_volume': '20', 'roast_type': 'mild', 'temperature': '205', 'grind_size': 'medium', 'user_id': '2'}
2FCZcqA6kCvZBObpWJ8r => {'rating': '7', 'grind_size': 'larger', 'water_volume': '18', 'roast_type': 'mild', 'user_id': '2', 'temperature': '206', 'coffee_bean_type': 'robusta', 'cup_size': '8', 'strength': '3'}
2gR2sWqeJVm8PBe7swvX => {'rating': '', 'strength': '', 'grind_size': '', 'cup_size': '', 'water_volume': '', 'roast_type': '', 'user_id': '', 'temper

# Clean Data

In [5]:
data.head() #this is to see the top 5 instances to see what i pulled from the database

,rating,strength,grind_size,cup_size,water_volume,user_id,roast_type,temperature,coffee_bean_type
0,4,3,larger,8,14,6,medium,186,excelsa
1,10,2,smaller,16,18,5,medium,190,robusta
2,9,3,medium,8,20,2,mild,205,robusta
3,7,3,larger,8,18,2,mild,206,robusta
4,,,,,,,,,


In [6]:
#this replaces all of the string values of coffee bean type and converting it to float values
data['coffee_bean_type'].loc[data['coffee_bean_type'] == 'robusta'] = 1.0 #converting robusta to 1
data['coffee_bean_type'].loc[data['coffee_bean_type'] == 'arabica'] = 2.0 #converting arabica to 2
data['coffee_bean_type'].loc[data['coffee_bean_type'] == 'liberica'] = 3.0 #converting liberica to 3
data['coffee_bean_type'].loc[data['coffee_bean_type'] == 'excelsa'] = 4.0 #converting excelsa to 4

#this replaces all of the string values of grind size and converting it to float values
data['grind_size'].loc[data['grind_size'] == 'medium'] = 0.0 #converting medium to 0
data['grind_size'].loc[data['grind_size'] == 'larger'] = 1.0 #converting larger to 1
data['grind_size'].loc[data['grind_size'] == 'smaller'] = 2.0 #converting smaller to 2

#this replaces all of the string values of roast type and converting it to float values
data['roast_type'].loc[data['roast_type'] == 'mild'] = 0.0 #converting mild to 0
data['roast_type'].loc[data['roast_type'] == 'medium'] = 1.0 #converting medium to 1
data['roast_type'].loc[data['roast_type'] == 'mediumdark'] = 2.0 #converting mediumdark to 2
data['roast_type'].loc[data['roast_type'] == 'dark'] = 3.0 #converting dark to 3

data.head() #calling top 5 instances of dataframe to check that I called it correctly

,rating,strength,grind_size,cup_size,water_volume,user_id,roast_type,temperature,coffee_bean_type
0,4,3,1.0,8,14,6,1.0,186,4.0
1,10,2,2.0,16,18,5,1.0,190,1.0
2,9,3,0.0,8,20,2,0.0,205,1.0
3,7,3,1.0,8,18,2,0.0,206,1.0
4,,,,,,,,,


In [7]:
#converts all strings/objects into floats
data = data._convert(numeric=True)

#deletes all non-null responses
data.dropna(subset=['water_volume', 'strength', 'cup_size', 'grind_size', 'coffee_bean_type', 'user_id', 'temperature', 'roast_type', 'rating'], inplace=True)

#checking dataframe for there's any non-null responses
data.info() 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 0 to 193
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rating            179 non-null    float64
 1   strength          179 non-null    float64
 2   grind_size        179 non-null    float64
 3   cup_size          179 non-null    float64
 4   water_volume      179 non-null    float64
 5   user_id           179 non-null    float64
 6   roast_type        179 non-null    float64
 7   temperature       179 non-null    float64
 8   coffee_bean_type  179 non-null    float64
dtypes: float64(9)
memory usage: 14.0 KB


# Test Random Forest Classifier Model

In [8]:
def rfc(x,y): #this is a function to call the Random Forest Classifier
    #splits independent and dependent variables into a test size & randomized values 
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
    
    #calls Random Forest Classifier model function
    clf = RandomForestClassifier()
    
    #model fits independent and dependent train values
    clf.fit(x_train, y_train) 
    
    #model predicts with independent test values
    y_pred = clf.predict(x_test)

    #outputs accuracy score of dependent test values and predicted independent values
    #accuracy score: calculate the accuracy of faction of correct prediction
    #(TP+TN)/ (TP+FN+TN+FP)
    print('accuracy =', accuracy_score(y_test, y_pred))
    
    #outputs f1 score of dependent test values and predicted independent values
    #f1 combines the precision and recall of a classifier into a single metric by taking their harmonic mean
    #best value at 1 and worst score at 0
    print('f1 score =', f1_score(y_test, y_pred, average='macro'))
    
    #outputs precision of dependent test values and predicted independent values
    #precision =  ratio tp / (tp + fp)
    print("Precision: ",precision_score(y_test, y_pred, average='macro'))
    
    #outputs classification report of dependent test values and predicted independent values
    print('Classification Report')
    print(classification_report(y_test, y_pred))
    
    #outputs confusion matrix of dependent test values and predicted independent values
    print('Confusion Matrix')
    print(confusion_matrix(y_test, y_pred))

In [9]:
#rfc function takes in independent variable values and dependent variable values
#calling rfc function and seeing affect from grind size
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['grind_size']].values)

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


accuracy = 0.4444444444444444
f1 score = 0.43939393939393945
Precision:  0.45042735042735044
Classification Report
              precision    recall  f1-score   support

         0.0       0.38      0.56      0.45         9
         1.0       0.47      0.54      0.50        13
         2.0       0.50      0.29      0.36        14

    accuracy                           0.44        36
   macro avg       0.45      0.46      0.44        36
weighted avg       0.46      0.44      0.44        36

Confusion Matrix
[[5 2 2]
 [4 7 2]
 [4 6 4]]


In [10]:
#calling rfc function and seeing affect from temperature
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['temperature']].values)

accuracy = 0.05555555555555555
f1 score = 0.02314814814814815
Precision:  0.01875
Classification Report
              precision    recall  f1-score   support

       180.0       0.00      0.00      0.00         2
       181.0       0.00      0.00      0.00         0
       182.0       0.20      1.00      0.33         1
       183.0       0.00      0.00      0.00         2
       184.0       0.00      0.00      0.00         4
       185.0       0.25      0.20      0.22         5
       186.0       0.00      0.00      0.00         2
       187.0       0.00      0.00      0.00         0
       188.0       0.00      0.00      0.00         5
       189.0       0.00      0.00      0.00         1
       190.0       0.00      0.00      0.00         1
       191.0       0.00      0.00      0.00         0
       193.0       0.00      0.00      0.00         0
       194.0       0.00      0.00      0.00         2
       195.0       0.00      0.00      0.00         1
       196.0       0.00      0.

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are 

In [11]:
#calling rfc function and seeing affect from water volume
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['water_volume']].values)

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


accuracy = 0.1111111111111111
f1 score = 0.06817391304347828
Precision:  0.0875
Classification Report
              precision    recall  f1-score   support

        12.0       0.00      0.00      0.00         3
        14.0       0.00      0.00      0.00         2
        16.0       0.00      0.00      0.00         7
        18.0       0.06      0.11      0.08         9
        20.0       0.38      0.20      0.26        15

    accuracy                           0.11        36
   macro avg       0.09      0.06      0.07        36
weighted avg       0.17      0.11      0.13        36

Confusion Matrix
[[ 0  1  2  0  0]
 [ 0  0  0  1  1]
 [ 0  2  0  4  1]
 [ 1  1  3  1  3]
 [ 0  2  0 10  3]]


In [12]:
#calling rfc function and seeing affect from rating
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['rating']].values)

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


accuracy = 0.08333333333333333
f1 score = 0.06262626262626263
Precision:  0.05416666666666666
Classification Report
              precision    recall  f1-score   support

         1.0       0.25      0.20      0.22         5
         2.0       0.00      0.00      0.00         2
         3.0       0.00      0.00      0.00         4
         4.0       0.00      0.00      0.00         4
         5.0       0.12      0.33      0.18         3
         6.0       0.00      0.00      0.00         6
         7.0       0.00      0.00      0.00         5
         8.0       0.17      0.33      0.22         3
         9.0       0.00      0.00      0.00         3
        10.0       0.00      0.00      0.00         1

    accuracy                           0.08        36
   macro avg       0.05      0.09      0.06        36
weighted avg       0.06      0.08      0.06        36

Confusion Matrix
[[1 0 0 0 2 0 2 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [1 0 0 1 0 1 1 0 0 0]
 [0 0 0 0 2 0 1 1 0 0]
 [0 0 0 1 1 1 0 

In [13]:
#calling rfc function and seeing affect from strength
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['strength']].values)

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


accuracy = 0.5
f1 score = 0.5117750804025314
Precision:  0.5527777777777777
Classification Report
              precision    recall  f1-score   support

         1.0       0.62      0.56      0.59         9
         2.0       0.33      0.60      0.43        10
         3.0       0.70      0.41      0.52        17

    accuracy                           0.50        36
   macro avg       0.55      0.52      0.51        36
weighted avg       0.58      0.50      0.51        36

Confusion Matrix
[[5 3 1]
 [2 6 2]
 [1 9 7]]


In [14]:
#calling rfc function and seeing affect from rating
rfc(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['rating']].values)

C:\Users\cindy\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


accuracy = 0.1388888888888889
f1 score = 0.11190476190476191
Precision:  0.11111111111111112
Classification Report
              precision    recall  f1-score   support

         1.0       0.50      0.40      0.44         5
         2.0       0.00      0.00      0.00         2
         3.0       0.33      0.25      0.29         4
         4.0       0.00      0.00      0.00         4
         5.0       0.17      0.33      0.22         3
         6.0       0.00      0.00      0.00         6
         7.0       0.00      0.00      0.00         5
         8.0       0.11      0.33      0.17         3
         9.0       0.00      0.00      0.00         3
        10.0       0.00      0.00      0.00         1

    accuracy                           0.14        36
   macro avg       0.11      0.13      0.11        36
weighted avg       0.13      0.14      0.13        36

Confusion Matrix
[[2 0 0 0 1 0 2 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [0 0 1 0 0 1 1 1 0 0]
 [0 0 0 0 2 0 1 1 0 0]
 [0 0 0 1 1 1 0 0

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classifi

# Gaussian Naive Baives Model

In [15]:
def GaussModel(x,y):
    #splits independent and dependent variables into a test, train, and validate values 
    #first splitting 60% out for training then 40% for validation & testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
    
    #then splitting the 40% into 20% validation & 20% testing
    x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.25, random_state=42)
    
    clf = GaussianNB() #variable holding Gaussian Naive Bayes model 
    clf.fit(x_train, y_train) #model fitting the training independent and dependent datasets
    y_pred = clf.predict(x_validate) #model predict the validate independent dataset

    #outputs accuracy score of dependent validate values and predicted independent values
    #accuracy score: calculate the accuracy of faction of correct prediction
    # (TP+TN)/ (TP+FN+TN+FP)
    print('accuracy =', accuracy_score(y_validate, y_pred))
    
    #outputs f1 score of dependent validate values and predicted independent values
    #f1 combines the precision and recall of a classifier into a single metric by taking their harmonic mean
    #best value at 1 and worst score at 0
    print('f1 score =', f1_score(y_validate, y_pred, average='macro'))

    #outputs precision of dependent validate values and predicted independent values
    #precision =  ratio tp / (tp + fp)
    print("Precision: ",precision_score(y_validate, y_pred, average='macro'))
    
    #outputs classification report of dependent validate values and predicted independent values
    print('Classification Report', classification_report(y_validate, y_pred))
    
    #outputs confusion matrix of dependent validate values and predicted independent values
    print('Confusion Matrix')
    print(confusion_matrix(y_validate, y_pred))

# K-Nearest Neighbors Model

In [16]:
def knnModel(x,y):
    #first splitting 60% out for training then 40% for validation & testing
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42) 
    #then splitting the 40% into 20% validation & 20% testing
    x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.25, random_state=42) 
    
    knn = neighbors.KNeighborsClassifier(n_neighbors=5) #set K Nearest Neighbors model as variable 
    knn.fit(x_train, y_train) #fitting training independent and dependent data 
    y_pred = knn.predict(x_validate) #model predict independent validation dataset values

    #outputs accuracy score of dependent validate values and predicted independent values
    #accuracy score: calculate the accuracy of faction of correct prediction
    #(TP+TN)/ (TP+FN+TN+FP)
    print('accuracy =', accuracy_score(y_validate, y_pred))
    
    #outputs f1 score of dependent validate values and predicted independent values
    #f1 combines the precision and recall of a classifier into a single metric by taking their harmonic mean
    #best value at 1 and worst score at 0
    print('f1 score =', f1_score(y_validate, y_pred, average='macro'))
    
    #outputs precision of dependent validate values and predicted independent values
    #precision =  ratio tp / (tp + fp)
    print("Precision: ",precision_score(y_validate, y_pred, average='macro'))
    
    #outputs classification report of dependent validate values and predicted independent values
    print('Classification Report')
    print(classification_report(y_validate, y_pred))
    
    #outputs confusion matrix of dependent validate values and predicted independent values
    print('confusion matrix')
    print(confusion_matrix(y_validate, y_pred))

In [22]:
print('KNN Model')
knnModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['rating']].values)

KNN Model
accuracy = 0.1388888888888889
f1 score = 0.09371794871794872
Precision:  0.0690079365079365
Classification Report
              precision    recall  f1-score   support

         1.0       0.22      0.67      0.33         3
         2.0       0.00      0.00      0.00         4
         3.0       0.14      0.33      0.20         3
         4.0       0.00      0.00      0.00         2
         5.0       0.20      0.33      0.25         3
         6.0       0.00      0.00      0.00         2
         7.0       0.00      0.00      0.00         6
         8.0       0.00      0.00      0.00         6
         9.0       0.12      0.20      0.15         5
        10.0       0.00      0.00      0.00         2

    accuracy                           0.14        36
   macro avg       0.07      0.15      0.09        36
weighted avg       0.06      0.14      0.09        36

confusion matrix
[[2 0 1 0 0 0 0 0 0 0]
 [2 0 0 0 0 0 0 1 1 0]
 [0 0 1 0 1 0 0 1 0 0]
 [0 0 2 0 0 0 0 0 0 0]
 [1 1 0 

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision 

In [18]:
print('Gaussian Model')
GaussModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['grind_size']].values)
print('KNN Model')
knnModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['grind_size']].values)

Gaussian Model
accuracy = 0.4722222222222222
f1 score = 0.4587590464027245
Precision:  0.6228070175438597
Classification Report               precision    recall  f1-score   support

         0.0       1.00      0.23      0.38        13
         1.0       0.37      0.88      0.52         8
         2.0       0.50      0.47      0.48        15

    accuracy                           0.47        36
   macro avg       0.62      0.52      0.46        36
weighted avg       0.65      0.47      0.45        36

Confusion Matrix
[[3 4 6]
 [0 7 1]
 [0 8 7]]
KNN Model
accuracy = 0.4722222222222222
f1 score = 0.47603121516165
Precision:  0.48484848484848486
Classification Report
              precision    recall  f1-score   support

         0.0       0.60      0.46      0.52        13
         1.0       0.40      0.75      0.52         8
         2.0       0.45      0.33      0.38        15

    accuracy                           0.47        36
   macro avg       0.48      0.51      0.48        3

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [19]:
print('Gaussian Model')
GaussModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['water_volume']].values)
print('KNN Model')
knnModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['water_volume']].values)

Gaussian Model
accuracy = 0.3055555555555556
f1 score = 0.1487179487179487
Precision:  0.14285714285714285
Classification Report               precision    recall  f1-score   support

        12.0       0.00      0.00      0.00         2
        14.0       0.00      0.00      0.00         6
        16.0       0.00      0.00      0.00         6
        18.0       0.29      0.73      0.41        11
        20.0       0.43      0.27      0.33        11

    accuracy                           0.31        36
   macro avg       0.14      0.20      0.15        36
weighted avg       0.22      0.31      0.23        36

Confusion Matrix
[[0 0 0 2 0]
 [0 0 1 5 0]
 [0 0 0 5 1]
 [0 0 0 8 3]
 [0 0 0 8 3]]
KNN Model
accuracy = 0.19444444444444445
f1 score = 0.09915966386554623
Precision:  0.07717391304347826
Classification Report
              precision    recall  f1-score   support

        12.0       0.00      0.00      0.00         2
        14.0       0.00      0.00      0.00         6
        16

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision 

In [20]:
print('Gaussian Model')
GaussModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['temperature']].values)
print('KNN Model')
knnModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['temperature']].values)

Gaussian Model
accuracy = 0.1388888888888889
f1 score = 0.038906695156695155


C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and

Precision:  0.026785714285714284
Classification Report               precision    recall  f1-score   support

       181.0       0.00      0.00      0.00         1
       182.0       0.00      0.00      0.00         2
       183.0       0.12      1.00      0.22         1
       184.0       0.00      0.00      0.00         3
       185.0       0.00      0.00      0.00         2
       187.0       0.38      0.60      0.46         5
       188.0       0.00      0.00      0.00         2
       189.0       0.00      0.00      0.00         1
       190.0       0.00      0.00      0.00         4
       191.0       0.14      1.00      0.25         1
       192.0       0.00      0.00      0.00         0
       193.0       0.00      0.00      0.00         1
       195.0       0.00      0.00      0.00         1
       197.0       0.00      0.00      0.00         1
       198.0       0.00      0.00      0.00         1
       199.0       0.00      0.00      0.00         1
       200.0       0.00   

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\metrics\_classification.p

In [21]:
print('Gaussian Model')
GaussModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['strength']].values)
print('KNN Model')
knnModel(data[['user_id','roast_type','cup_size','coffee_bean_type']].values, data[['strength']].values)

Gaussian Model
accuracy = 0.4444444444444444
f1 score = 

C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\utils\validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\cindy\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


0.41007905138339923
Precision:  0.4621693121693122
Classification Report               precision    recall  f1-score   support

         1.0       0.71      0.31      0.43        16
         2.0       0.45      0.69      0.55        13
         3.0       0.22      0.29      0.25         7

    accuracy                           0.44        36
   macro avg       0.46      0.43      0.41        36
weighted avg       0.52      0.44      0.44        36

Confusion Matrix
[[5 7 4]
 [1 9 3]
 [1 4 2]]
KNN Model
accuracy = 0.4722222222222222
f1 score = 0.4297979797979798
Precision:  0.43092691622103385
Classification Report
              precision    recall  f1-score   support

         1.0       0.59      0.62      0.61        16
         2.0       0.45      0.38      0.42        13
         3.0       0.25      0.29      0.27         7

    accuracy                           0.47        36
   macro avg       0.43      0.43      0.43        36
weighted avg       0.47      0.47      0.47        